In [ ]:
import os
import tensorflow as tf
import zipfile

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
os.listdir('../input/resnet50')

In [ ]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train['id_code'] = train['id_code'] + '.png'

In [ ]:
import cv2 as cv

In [ ]:
plt.figure(figsize=[20,20])
pos = 1
for file in train['id_code'][:25]:
    img = cv.imread('../input/aptos2019-blindness-detection/train_images/'+file)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (1024, 1024))
    plt.subplot(5, 5, pos)
    pos += 1
    plt.title(train['diagnosis'][pos])
    plt.imshow(img)
plt.show()

In [ ]:
sns.countplot(x='diagnosis', data=train, palette="GnBu_d")

In [ ]:
# class_0, class_1, class_2, class_3, class_4 = train['diagnosis'].value_counts()
# df_0 = train[train['diagnosis'] == '0']
# df_1 = train[train['diagnosis'] == '1']
# df_2 = train[train['diagnosis'] == '2']
# df_3 = train[train['diagnosis'] == '3']
# df_4 = train[train['diagnosis'] == '4']

In [ ]:
train['diagnosis'].value_counts()

In [ ]:
# df_0_u = df_0.sample(class_4)
# df_1_u = df_1.sample(class_4)
# df_2_u = df_2.sample(class_4)
# df_4_u = df_4.sample(class_4)

In [ ]:
# df_u = pd.concat([df_0_u, df_1_u, df_2_u, df_3, df_4_u], axis=0)

In [ ]:
# sns.countplot(x='diagnosis', data=df_u)

In [ ]:
# df_u['diagnosis'] = df_u['diagnosis'].astype('str')
# # train['diagnosis'] = train['diagnosis'].astype('str')

In [ ]:
seed = 10
batch_size = 32
img_size = 32
nb_epochs = 5

In [ ]:
train = train.sample(frac=1, random_state=seed)

In [ ]:
train['diagnosis'] = train['diagnosis'].astype('str')

In [ ]:
x = train['id_code'].values
y = train['diagnosis'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_va, y_train, y_val = train_test_split(x, y, random_state = 0, test_size=0.2, stratify=y)

In [ ]:
x_train.shape, x_val.shape, x_test.shape

In [ ]:
train_df = pd.DataFrame({'image': x_train, 'class': y_train})
valid_df = pd.DataFrame({'image': x_val, 'class': y_val})

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
  dataframe = train_df,
    directory = '../input/aptos2019-blindness-detection/train_images',
    target_size=(img_size,img_size),
    x_col='image',
    y_col='class',
    class_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
valid_generator = valid_datagen.flow_from_dataframe(
  dataframe = valid_df,
    directory = '../input/aptos2019-blindness-detection/train_images',
    target_size=(img_size, img_size),
    x_col='image',
    y_col='class',
    class_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
# local_weights_file = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:

# pre_trained_model = tf.keras.applications.resnet50.ResNet50(input_shape = (256, 256, 1), 
#                                 include_top = False, 
#                                 weights = None)

In [ ]:
# pre_trained_model.load_weights(local_weights_file)

In [ ]:
# for layer in pre_trained_model.layers:
#   layer.trainable = False

In [ ]:
# pre_trained_model.summary()

In [ ]:
# last_layer = pre_trained_model.get_layer('mixed7')
# print('last layer output shape: ', last_layer.output_shape)
# last_output = last_layer.output

In [ ]:
model = tf.keras.models.Sequential([
#     pre_trained_model,
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Flatten(),
    tf.keras.layers.Conv2D(32, (6, 6), activation = tf.nn.relu, input_shape = (img_size, img_size, 1)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation = tf.nn.softmax)
])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("model_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.000001)

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator, 
                    epochs=nb_epochs, 
                    validation_data=valid_generator, 
                    callbacks=[learning_rate_reduction, checkpoint],
                    steps_per_epoch=100,
                    use_multiprocessing=True,
                    class_weight={0:0.1, 1: 0.2, 2:0.2, 3:0.3, 4:0.2})

In [ ]:
# # Plot the loss and accuracy curves for training and validation 
# fig, ax = plt.subplots(2,1)
# ax[0].plot(history.history['loss'], color='b', label="Training loss")
# ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
# legend = ax[0].legend(loc='best', shadow=True)

# ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
# ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
# legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
import numpy as np

In [ ]:
# train_preds = model.predict_generator(valid_generator)

In [ ]:
# train_preds = [np.argmax(pred) for pred in train_preds]

In [ ]:
# from sklearn.metrics import cohen_kappa_score

In [ ]:
# print("Train Cohen Kappa score: %.3f" % cohen_kappa_score(train_preds, y_test.astype('int'), weights='quadratic'))

In [ ]:
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
test_df = pd.DataFrame()

In [ ]:
test_df['image'] = test['id_code'] + '.png'

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
  dataframe = test_df,
    directory = '../input/aptos2019-blindness-detection/test_images',
    target_size=(img_size, img_size),
    x_col='image',
    y_col=None,
    class_mode=None,
    color_mode='grayscale',
    batch_size=batch_size
)

In [ ]:
pred = model.predict_generator(test_generator)

In [ ]:
import numpy as np

In [ ]:
pred = np.argmax(pred, 1).reshape(-1, 1)

In [ ]:
pred.shape, test.shape

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
submission = test.copy()

In [ ]:
submission['diagnosis'] = pred

In [ ]:
sns.countplot(x='diagnosis', data=submission)

In [ ]:
submission.to_csv('submission.csv',index=False)